# Xây dựng word2vec model bằng Gensim

Now that we have understood how word2vec model works, let us see how to build word2vec model using gensim library. Gensim is one of the popular scientific software packages widely used for building vector space models. It can be easily installed via pip. So, we can just type the following command in our terminal to install the gensim library:

pip install -U gensim

Now, we will learn how to build word2vec model using gensim. 

In [1]:
import warnings
warnings.filterwarnings('ignore')

# data processing
import pandas as pd
import re
from nltk.corpus import stopwords
stopWords = stopwords.words('english')

# modelling
from gensim.models import Word2Vec
from gensim.models import Phrases
from gensim.models.phrases import Phraser

## Đọc dữ liệu
/> Đọc dữ liệu được lưu trong file `text.csv`, có kích thước khoảng 23 MB.

In [2]:
data = pd.read_csv('data/text.csv', header=None)

/> Xem qua vài quan sát trong dữ liệu.

In [3]:
data.head()

,0
0,room kind clean strong smell dogs. generally a...
1,stayed crown plaza april april . staff friendl...
2,booked hotel hotwire lowest price could find. ...
3,stayed husband sons way alaska cruise. loved h...
4,girlfriends stayed celebrate th birthdays. pla...


## Tiền xử lí dữ liệu
/> Định nghĩa hàm dùng để tiền xử lí dữ liệu.

In [4]:
def pre_process(text):    
    '''
    Lowercase text '''
    text = str(text).lower()
    
    ''' 
    Remove các kí tự đặc biệt, số, khoảng trắng thừa '''
    text = re.sub(r'[^A-Za-z0-9\s.]', r'', text)
    
    '''
    Remove new line characters '''
    text = re.sub(r'\n',r' ',text)
    
    '''
    Remove stopwords '''
    text = " ".join([word for word in text.split() if word not in stopWords])
    
    return text

/> Thử áp dụng hàm `pre_process()` lên một mẫu dữ liệu.

In [5]:
pre_process(data[0][50])

'agree fancy. everything needed. breakfast pool hot tub nice shuttle airport later checkout time. noise issue tough sleep through. awhile forget noisy door nearby noisy guests. complained management later email credit compd us amount requested would return.'

/> Pre-processing trên toàn bộ data.

In [6]:
data[0] = data[0].map(lambda x: pre_process(x))

/> Xem lại dữ liệu sau khi pre-processing.

In [7]:
data[0].head()

0    room kind clean strong smell dogs. generally a...
1    stayed crown plaza april april . staff friendl...
2    booked hotel hotwire lowest price could find. ...
3    stayed husband sons way alaska cruise. loved h...
4    girlfriends stayed celebrate th birthdays. pla...
Name: 0, dtype: object

/> Genism library yêu cầu input có dạng list of lists. Mỗi quan sát trong dữ liệu là một set of sentences. Cho nên ta cần tách các sentence bằng dấu '.' sau đó bỏ chúng vào list. 
  ```python
  text = [[word1, word2, word3], [word1, word2, word3]]
  ```

In [8]:
data[0][1].split('.')[:5]

['stayed crown plaza april april ',
 ' staff friendly attentive',
 ' elevators tiny ',
 ' food restaurant delicious priced little high side',
 ' course washington dc']

/> Chúng ta đã có một data là một list of sentence. Bây giờ chúng ta cần tách các sentence thành list of words dựa vào kí tự ' '.

In [9]:
corpus = []
for line in data[0][1].split('.'):
    words = [x for x in line.split()]
    corpus.append(words)

/> Bây giờ, input của chúng ta đã có dạng chuẩn của Gensim yêu cầu là list of lists.

In [10]:
corpus[:2]

[['stayed', 'crown', 'plaza', 'april', 'april'],
 ['staff', 'friendly', 'attentive']]

/> Các đoạn code trên chúng ta chỉ thử trên một sentence, bây giờ chúng ta sẽ chuyển đổi toàn bộ dataset.

In [11]:
data = data[0].map(lambda x: x.split('.'))

corpus = []
for i in (range(len(data))):
    for line in data[i]:
        words = [x for x in line.split()]
        corpus.append(words)

corpus[:2]

[['room', 'kind', 'clean', 'strong', 'smell', 'dogs'],
 ['generally', 'average', 'ok', 'overnight', 'stay', 'youre', 'fussy']]

In [12]:
corpus[111]

['connected', 'rivercenter', 'mall', 'downtown', 'san', 'antonio']

/> Giả sử chúng ta có từ 'new york', chúng ta sẽ thêm dấu gạch '_' để thay cho khoảng trắng. Chúng ta thiết lập `min_count=25`, tức ta bỏ qua tất cả các từ có tần số xuất hiện dưới 25.

In [13]:
phrases = Phrases(sentences=corpus, min_count=25, threshold=50)
bigram = Phraser(phrases)

In [14]:
for index,sentence in enumerate(corpus):
    corpus[index] = bigram[sentence]

/> Kiểm tra dữ liệu.

In [15]:
corpus[111]

['connected', 'rivercenter', 'mall', 'downtown', 'san_antonio']

In [15]:
corpus[9]

['course', 'washington_dc']

## Build Model

/> Bây giờ chúng ta sẽ build model word2vec, trước tiên chúng ta cần xác định một vài hyperparams.
* `vector_size`: kích thước của embedding vector. Tùy chỉnh dựa vào kích thước của dataset mà ta có.
* `window_size`: kích thước của cửa sổ context words.
* `min_count`: tần số thấp nhất của từ trong dataset.
* `workers`: chỉ định số nhân CPU dùng để training model. 
* `sg`: bằng `1` sẽ dùng skip-gram method để training, nếu `0` thì chỉ định CBOW method để training.

In [21]:
vector_size = 100
window_size = 2
epochs = 100
min_count = 2
workers = 4
sg = 1

/> Training model.

In [23]:
model = Word2Vec(corpus, sg=sg, window=window_size, vector_size=vector_size, min_count=min_count, workers=workers, epochs=epochs)

/> Lưu lại word2vec model.

In [24]:
model.save('./model/word2vec.model')

/> Load word2vec model.

In [25]:
model = Word2Vec.load('./model/word2vec.model')

## Đánh giá word2vec model
/> Gensim cung cấp phương thức `most_similar()` sẽ cho ta biết những từ nào tương đồng với một từ được cung cấp. Dưới đây từ `san_deigo` được cung cấp là đầu vào, ta cần tìm các từ mà có sự tương đồng với từ này.

In [27]:
model.wv.most_similar('san_diego')

[('san_antonio', 0.7916811108589172),
 ('san_francisco', 0.7779590487480164),
 ('memphis', 0.7521619200706482),
 ('austin', 0.7430611848831177),
 ('seattle', 0.7425510287284851),
 ('dallas', 0.7392098903656006),
 ('boston', 0.7264854907989502),
 ('la', 0.7237415313720703),
 ('dc', 0.7234782576560974),
 ('phoenix', 0.7193017601966858)]

/> Có thể áp dụng các phép toán số học trên vector, ví dụ:
  $$woman + king - man = queen$$

In [28]:
model.wv.most_similar(positive=['woman', 'king'], negative=['man'], topn=1)

[('queen', 0.7051153182983398)]

/> Chúng ta có thể tìm hiểu xem từ nào là không phù hợp với đa số các từ được cung cấp trong list.

In [29]:
text = ['los_angeles','indianapolis', 'holiday', 'san_antonio','new_york']
model.wv.doesnt_match(text)

'holiday'